In [1]:
from contextlib import contextmanager
import os
import datetime as dt
import numpy as np
import h5py
import time
import sys
os.chdir('../telescope_control/data_aquisition/')
sys.path.append('../')
sys.path.append('/software_git_repos/cofe-python-analysis-tools/utils_meinhold/')
sys.path.append('/software_git_repos/cofe-python-analysis-tools/utils_zonca/')
from glob import glob

sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../VtoT/')


import Tkinter,tkFileDialog

import cofe_util as cu
import realtime_gp as rt
import datetime

from Tkinter import *
import ttk

In [2]:
def nametochan(name):
    #function to convert channel numbers to channel names

    #names of each channel
    chans = {
        'all': 'all',  'H1HiAC':'ch0',  'H1HiDC':'ch1',
        'H1LoAC':'ch2' ,  'H1LoDC':'ch3', 'H2HiAC':'ch4' ,
        'H2HiDC':'ch5' ,  'H2LoAC':'ch6',  'H2LoDC':'ch7',
        'H3HiAC':'ch8', 'H3HiDC':'ch9',  'H3LoAC':'ch10',
         'H3LoDC':'ch11', 'HornTop':'ch12', 'Amplifier': 'ch13',
        'Cooler':'ch14', 'Transition':'ch15'}

    chan = chans[name]

    return chan

In [3]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Widget', 'Text', 'Button']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [10]:
def plot_some_chans2(datatype='demod',plottype='toi',filelist=None,component='T',samprate=30,minfreq=.1):
    
    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    main = Tk()
    main.title("Choose channels to plot")
    main.geometry("+50+150")
    frame = ttk.Frame(main, padding=(3, 3, 12, 12))
    frame.grid(column=0, row=0, sticky=(N, S, E, W))

    #set up the listbox to choose what chans to plot
    chan_labels = StringVar()
    chan_labels.set("H1HiAC H1HiDC H1LoAC H1LoDC H2HiAC H2HiDC H2LoAC H2LoDC\
                 H3HiAC H3HiDC H3LoAC H3LoDC HornTop Amplifier Cooler Transition")           
    tlist=[]
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            h=np.float64(f[-11:-9])
            m=np.float64(f[-9:-7])
            s=np.float64(f[-7:-5])
            hf=h5py.File(f)
            t=h+m/60.+(s+(hf['demod_data']['rev']-hf['demod_data']['rev'][0])/1000.)/3600.
            tlist.append(t)  #just generate a time to plot relative to
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        h=np.float64(filelist[0][-11:-9])
        m=np.float64(filelist[0][-9:-7])
        s=np.float64(filelist[0][-7:-5])
        d=rt.demod.read_raw(filelist)
        t=h+m/60.+(s+(d['rev']-d['rev'][0])/1000.)/3600.
        tlist.append(t)
    ut=concatenate(tlist)
    #figure()
    #plot(ut)
    lstbox = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox.grid(column=0, row=0, columnspan=2)
    
    lstbox2 = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox2.grid(column=2, row=0, columnspan=2)
    #start the plot

    fname=filelist[0]
    yyyy=os.path.dirname(filelist[0])[-8:-4]
    mm=os.path.dirname(filelist[0])[-4:-2]
    dd=os.path.dirname(filelist[0])[-2:]

    def select():
        reslist = list()
        selection = lstbox.curselection()
        figure()
        if datatype=='demod':
            title('%s TOI from %s %s %s' %(component,mm,dd,yyyy))
        if datatype=='raw':
            title('TOI from %s %s %s' %(mm,dd,yyyy))

        for i in selection:
            entry = lstbox.get(i)
            reslist.append(entry)
        for val in reslist:
            chan=nametochan(val)
            if datatype=='demod':
                plot(ut,d[chan][component],label=val)
            if datatype=='raw':
                plot(d[chan].flatten(),label=val)
            xlabel('Samples')
            ylabel('Output [v]')
            legend()

    def selectfft():
        reslist2 = list()
        selection2 = lstbox2.curselection()
        figure()
        if datatype=='demod':
            title('%s ASD from %s %s %s' %(component,mm,dd,yyyy))
        if datatype=='raw':
            title('ASD from %s %s %s' %(mm,dd,yyyy))
                

        for i in selection2:
            entry2 = lstbox2.get(i)
            reslist2.append(entry2)
        for val in reslist2:
            chan=nametochan(val)
            if datatype=='demod':
                freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
            if datatype=='raw':
                freq,psd=cu.nps(d[chan].flatten(),samprate*256,minfreq=minfreq)
            plot(freq,sqrt(psd)*1e9,label=val)
            xlabel('Frequency [Hz]')
            ylabel(r'ASD [$\frac{nV}{\sqrt{Hz}}$]')
            legend()
                
    btn = ttk.Button(frame, text="Select for TOI", command=select)
    btn.grid(column=1, row=1)
    
    btn2 = ttk.Button(frame, text="Select for ASD", command=selectfft)
    btn2.grid(column=3, row=1)
    
    main.mainloop()

In [5]:
def read_some_data(datadict,datalabel,datatype='demod',filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            hf=h5py.File(f)
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        d=rt.demod.read_raw(filelist)
    datadict[datalabel]=d
    return

In [6]:
def asd_from_dict(data_dict,minfreq=1.):
    #run nps on all the channels for each of the loaded data sets, return asd dictionary
    asd_dict={}
    chans=['ch%s' %str(i) for i in range(16)]
    for datset in data_dict.keys():
        asd_dict[datset]={}
        datatype='demod'
        print(datset)
        d=data_dict[datset]
        if len(d.dtype)==18:
            datatype='raw'
        samprate=1000./float(d['rev'][10]-d['rev'][9])
        for chan in chans:
            if datatype=='raw':
                freq,psd=cu.nps(d[chan].flatten(),samprate*256,minfreq=minfreq)
                asd_dict[datset]['freq']=freq
                asd_dict[datset][chan]=sqrt(psd)
            if datatype=='demod':
                asd_dict[datset][chan]={}
                for component in ['T','Q','U']:
                    freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
                    asd_dict[datset][chan][component]=sqrt(psd)
                    asd_dict[datset]['freq']=freq
                    
    return asd_dict            

In [7]:
def plot_asd_dict(asd_dict,chanlist=['H1HiAC'],componentlist=['T'],datalist=None):
    #function to give a choice of preloaded datasets from dictionary data_dict
    figure()
    if datalist==None:
        datalist=asd_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in datalist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    
    for dname in datalist:
        for channame,component in zip(chanlist,componentlist):
            try:
                asd=asd_dict[dname][nametochan(channame)][component] #if it's demod data so a dict
            except:
                asd=asd_dict[dname][nametochan(channame)]           #if it's raw data so not a dict
            plot(asd_dict[dname]['freq'],asd,label=dname+ '  '+channame+' '+component,lw=2)
    legend()
    xlabel('Frequency [Hz]')
    ylabel(r'ASD [$\frac{V}{\sqrt{Hz}}$]')

In [8]:
def plot_toi_dict(toi_dict,chanlist=['H1HiAC'],componentlist=['T'],datalist=None):
    #function to give a choice of preloaded datasets from dictionary data_dict
    figure()
    if datalist==None:
        datalist=asd_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in chanlist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    titletxt=[]
    for dname in datalist:
        samprate=1000./(toi_dict[dname]['rev'][1]-toi_dict[dname]['rev'][0])
        print(dname,samprate)
        titletxt.append(dname)
        for channame,component in zip(chanlist,componentlist):
            print(channame,component)
            try:
                toi=toi_dict[dname][nametochan(channame)][component] #if it's demod data so a dict
                t=toi_dict[dname]['azi'][0]+arange(len(toi))/samprate
                plot(t,toi,label=dname+ '  '+channame+' '+component,lw=2)
            except:
                toi=toi_dict[dname][nametochan(channame)].flatten()#if it's raw data so not a dict
                t=toi_dict[dname]['azi'][0]+arange(len(toi))/(256.*samprate)
                plot(t,toi,label=dname+ '  '+channame+' '+component,lw=2)
            
    title(titletxt)
    legend()
    xlabel('Time [S]')
    ylabel(r'Signl [V]')

In [13]:
plot_some_chans2(datatype='raw')

D:\software_git_repos\cofe-python-analysis-tools\utils_zonca\demod\datparsing.py:38: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 256000 but corresponding boolean dimension is 255999
  good[duplicates] = False


0
16
0
16
0
16
0
16
0
16
0
16
0
16
0
16
0
16


In [24]:
ddict={}

In [33]:
read_some_data(ddict,'LN_3_13',datatype='raw')

0
16
0
16
0
16


In [40]:
asd_dict=asd_from_dict(ddict, minfreq=10)

LN_3_13
amb_3_13
part_3_13


In [42]:
plot_asd_dict(asd_dict,chanlist=['H1HiAC'])

C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:17: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [148]:
plot_toi_dict(ddict,chanlist=['H3HiAC'],componentlist=['T'],datalist=['amb_3_12','sky_3_12','part_3_12'])

('amb_3_12', 29.411764705882351)
('H3HiAC', 'T')
('sky_3_12', 29.411764705882351)
('H3HiAC', 'T')
('part_3_12', 31.25)
('H3HiAC', 'T')


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [146]:
tsys=15
tamb=295
beta=.27
vsky=.24
vpart=.44
vamb=.9

g=vamb/(tsys+tamb)
print('g=',g,'v/k')
tsky=(vsky/g)-tsys
print('tsky=',tsky)
print('dtpart=',(tsky*(1-beta)+tamb*beta+tsys)*g)

('g=', 0.002903225806451613, 'v/k')
('tsky=', 67.66666666666666)
('dtpart=', 0.41820000000000007)


In [147]:
y=vamb/vsky
tsys=(tamb-y*tsky)/(y-1)
print(tsys)

15.0


In [155]:
.00087
print(1.4*.175*100/sqrt(4.5e9))

0.000365224436325
